In [1]:
from classes import *
from functools import reduce
import pygame

pygame 2.0.1 (SDL 2.0.14, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
def arrange(nbomb,space):
    if(nbomb==1):
        return frozenset(frozenset({i}) for i in space)
    return frozenset(frozenset([i]) | t for i in space for t in arrange(nbomb-1,space - {i}))


In [3]:
class Barrangement:
    def __init__(self,arrangement,whole):
        self.arrangement=arrangement
        self.whole=whole
    def __add__(self,B1):
        if(self.whole=={}): return B1
        if(B1.whole=={}): return self
        
        intersect= B1.whole & self.whole
        return Barrangement({b1 | b2 if(b1 & b2 == b1 & intersect == b2 & intersect) else None
        for b1 in B1.arrangement
        for b2 in self.arrangement}-{None},B1.whole | self.whole)
    def probability(self):
        prob={key:0 for key in self.whole}
        L=len(self.arrangement)
        if(L==0):
            return prob
        for S in self.arrangement:
            for key in S:
                prob[key]+=1

        for key in prob.keys():  
            prob[key]/=L
        return prob
    def __repr__(self):
        return f'From:    {self.whole} \nArrange: {self.arrangement}   '

class Psum:
    def __init__(self, arrangements):
        self.arrangements=arrangements
        self.Sets=[a.whole for a in arrangements]
    def probs(self):
        self.islands()
        Probs=[[Barrangement({},{})]]*len(self.Sets)
        for index,check in enumerate(self.Sets):
          for sample in self.arrangements:          
                if(sample.whole.issubset(check)):
                    Probs[index]=Probs[index]+[sample]
                    
        Probs=[reduce(lambda x,y:x+y,P).probability() for P in Probs]
        Prob = {}
        for P in Probs:
            Prob.update(P)
        return Prob
    
    def islands(self):
        if(len(self.Sets)==1):
            return
        for index,value in enumerate(self.Sets[1:]):
            if(self.Sets[0] & value != set()):
                self.Sets[0]=self.Sets[0] | self.Sets[index+1]
                del self.Sets[index+1]
                self.islands()
                return
        a0=self.Sets[0]
        del self.Sets[0]
        self.islands()
        
        self.Sets.append(a0)


In [4]:
def boardcolorinit(self):
    self.set_background_color((0,0,255))
    self.prev_board=self.visible
    self.Arrange=Barrangement({},{})
def update(self):
    Grid=self.visible
    positions=np.array(np.where((Grid >= 1) & (Grid < 9))).T
    neighbors=self.neighbors(positions)
    Cond=self.visible[tuple(neighbors.T)]
    neighbors=neighbors[(Cond==9) | (Cond==-5)]
    
    Neighborset={'all':{tuple(p) for p in neighbors}}    
    valueset={}
    for p in positions:
         neighbor = np.array([(p[0]+addx,p[1]+addy) for addx in range(-1,2) for addy in range(-1,2)])        
         neighbor = {tuple(p) for p in neighbor} & Neighborset['all']
         Neighborset[tuple(p)]=neighbor
         valueset[tuple(p)]=self.visible[p[0],p[1]]
    
    arrangements=[]
    for key in valueset.keys():
        neighbor=Neighborset[key]
        value=valueset[key]
        arrangements.append(Barrangement(arrange(value,neighbor),neighbor))
        
    Prob=Psum(arrangements).probs()
    for key in Prob.keys():
        n=int(Prob[key]*512)
        color=(min(n,255),min((512-n),255),0)
        self.background[key]=color

Grid.update=update
Grid.boardcolorinit=boardcolorinit

In [ ]:
G=Grid(80,45)
G.generate(800)

GG=Game(G,Width=1600,Height=900,border=20,top_border=50)
GG.set_background_color((127,127,127))
GG.Grid.set_background_color((0,0,255))
GG.draw_board()
GG.play()